In [8]:
import ee
import subprocess
from google.cloud import storage
from datetime import datetime
ee.Initialize()

In [22]:
bucket_name = 'opera-bucket-rtc'
folderPath = 'products/test-run'
targetCollectionPath = 'projects/opera-one/assets/RTC/systemTestVH'

In [23]:
targetCollection = ee.ImageCollection(targetCollectionPath)
targetIDs = targetCollection.aggregate_array('system:index').getInfo()
print(targetIDs)

['OPERA_L2_RTC-S1_T121-258596-IW1_20220411T000836Z_20230503T114347Z_S1A_30_v0_0_VH']


In [25]:
storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_name, prefix=folderPath+'/')
gcsKeys = []
for blob in blobs:
    gcsKeys.append(blob.name)
print(len(gcsKeys))

137


In [27]:
i = 0
skip=0
for key in gcsKeys:
    filename = key.split('/')[-1]
    gcsurl = f'gs://{bucket_name}/{key}'
    asset_name = filename.split('.tif')[0].replace('.','_')
    if asset_name in targetIDs:
        skip = skip+1
    else:
        s = filename.split('_')
        #print(s)
        tile = s[3]
        start_time = int(datetime.strptime(s[4],'%Y%m%dT%H%M%SZ').timestamp()*1000)
        prod_time = int(datetime.strptime(s[5],'%Y%m%dT%H%M%SZ').timestamp()*1000)
        sensor = s[6]
        band = s[-1].split('.')[0]
        ee_cmd = f'earthengine upload image --asset_id={targetCollectionPath}/{asset_name} --pyramiding_policy=mean --bands={band} -ts={start_time} -p=sensor={sensor} -p=prod_time={prod_time} -p=tile={tile} {gcsurl}'
        #print(ee_cmd)
        subprocess.run(ee_cmd,shell=True)
    i = i+1
    if i%10 == 0:
        print(f'started {i} ingestion tasks, skipped {skip} tasks')

Started upload task with ID: JHSGOXRV5VM7E4I5DJ6WLFL6
Started upload task with ID: EJMHRXRIAGXQR3BDLUDISZ6B
Started upload task with ID: M4NDI3YDLKBT74ZYIRI5B5W3
Started upload task with ID: 63KUBVVXCXM2VIYABGUWRYT6
Started upload task with ID: 7GCVWVTGLUPW6KECG2VMOU63
Started upload task with ID: HYY6HGGIDO3CU5DEMB6UNLZS
Started upload task with ID: TDEPXYCD4QSB4GMPGNWQHSCA
Started upload task with ID: S76GUUZM24QH557EL3DTFC5I
Started upload task with ID: 7MUPUXVVZDVTCFUEXC6PEJZO
started 10 ingestion tasks, skipped 1 tasks
Started upload task with ID: HXL2EUAMGJAKKCDNXCN3AOEL
Started upload task with ID: JBIXC5O4FVQ5Y6DCSYL3LL7W
Started upload task with ID: OXU4JJ45X7GHG2WVJNYKYD5I
Started upload task with ID: DVQY4JYWUWXPJZSCMQVMCUYJ
Started upload task with ID: YOOQCDIDDFEKP7WWUPCDQUTS
Started upload task with ID: AN6OC5OZBLG4LZZVWX2JCNWD
Started upload task with ID: 5BC6A3QEGRBAFVQXI5JSVILI
Started upload task with ID: 2SUKRCZGQHHRFM5LGG6TGD6F
Started upload task with ID: X262ITBB6